<h1>European Flights Analysis<h1>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px


df = pd.read_csv("data/flights.csv")
df

,YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_ICAO,APT_NAME,STATE_NAME,FLT_DEP_1,FLT_ARR_1,FLT_TOT_1,FLT_DEP_IFR_2,FLT_ARR_IFR_2,FLT_TOT_IFR_2,Pivot Label
0,2016,1,JAN,2016-01-01T00:00:00Z,EBAW,Antwerp,Belgium,4,3,7,NaN,NaN,NaN,Antwerp (EBAW)
1,2016,1,JAN,2016-01-01T00:00:00Z,EBBR,Brussels,Belgium,174,171,345,174.0,161.0,335.0,Brussels (EBBR)
2,2016,1,JAN,2016-01-01T00:00:00Z,EBCI,Charleroi,Belgium,45,47,92,45.0,45.0,90.0,Charleroi (EBCI)
3,2016,1,JAN,2016-01-01T00:00:00Z,EBLG,Liège,Belgium,6,7,13,NaN,NaN,NaN,Liège (EBLG)
4,2016,1,JAN,2016-01-01T00:00:00Z,EBOS,Ostend-Bruges,Belgium,7,7,14,NaN,NaN,NaN,Ostend-Bruges (EBOS)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688094,2022,5,MAY,2022-05-31T00:00:00Z,LYBE,Belgrade - Nikola Tesla,Serbia,80,85,165,NaN,NaN,NaN,Belgrade - Nikola Tesla (LYBE)
688095,2022,5,MAY,2022-05-31T00:00:00Z,LYPG,Podgorica,Montenegro,19,18,37,NaN,NaN,NaN,Podgorica (LYPG)
688096,2022,5,MAY,2022-05-31T00:00:00Z,LZIB,Bratislava,Slovakia,21,21,42,20.0,19.0,39.0,Bratislava (LZIB)
688097,2022,5,MAY,2022-05-31T00:00:00Z,UDYZ,Yerevan,Armenia,39,40,79,NaN,NaN,NaN,Yerevan (UDYZ)


<h2>Test Graph</h2>

In [2]:
# Conta il numero di voli per ogni stato e ordina dal più grande al più piccolo
df_sorted = df["STATE_NAME"].value_counts().reset_index()
df_sorted.columns = ["State", "NumFlights"]
df_sorted = df_sorted.sort_values(by="NumFlights", ascending=True)  # Ordinamento decrescente

# Creazione dell'istogramma ordinato
fig = px.histogram(df_sorted, x="NumFlights", y="State", nbins=1000, title="Num of Flights per State (Ordered)")

# Mantieni l'ordine corretto assegnando i valori di Y manualmente
fig.update_layout(
    height=1200,  # Ingrandisci il grafico verticalmente
    yaxis=dict(categoryorder="array", categoryarray=df_sorted["State"].tolist())  # Ordine manuale
)

# Mostra il grafico
fig.show()


In [3]:

# Conversione della data
df["FLT_DATE"] = pd.to_datetime(df["FLT_DATE"])

# Calcolo media voli giornalieri per mese
monthly_avg = df.groupby("MONTH_NUM")["FLT_TOT_1"].mean().reset_index()

# Aggiunta nomi mesi
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
monthly_avg["Month"] = monthly_avg["MONTH_NUM"].apply(lambda x: month_names[x - 1])

# Ordinamento corretto
monthly_avg = monthly_avg.sort_values("MONTH_NUM")

# Grafico con Plotly
fig = px.bar(
    monthly_avg,
    x="Month",
    y="FLT_TOT_1",
    title="Media dei voli giornalieri per mese (2016–2022)",
    labels={"FLT_TOT_1": "Voli giornalieri medi", "Month": "Mese"}
)

fig.update_layout(xaxis_title="Mese", yaxis_title="Voli giornalieri medi")
fig.show()


In [4]:

# Conversione della colonna FLT_DATE in datetime
df["FLT_DATE"] = pd.to_datetime(df["FLT_DATE"])

# Estrazione dell'anno
df["YEAR"] = df["FLT_DATE"].dt.year

# Calcolo della media dei voli giornalieri per anno
yearly_avg = df.groupby("YEAR")["FLT_TOT_1"].mean().reset_index()

# Creazione del grafico a linee con punti
fig = px.line(
    yearly_avg,
    x="YEAR",
    y="FLT_TOT_1",
    markers=True,
    title="Media dei voli giornalieri per anno (2016–2022)",
    labels={"FLT_TOT_1": "Voli giornalieri medi", "YEAR": "Anno"}
)

fig.update_layout(xaxis_title="Anno", yaxis_title="Voli giornalieri medi")
fig.show()

In [5]:
# Calcolo dei voli in arrivo totali per aeroporto
arrivals = df.groupby(["APT_ICAO", "APT_NAME"])["FLT_ARR_1"].sum().reset_index()

# Seleziona i primi 10 aeroporti
top10 = arrivals.sort_values("FLT_ARR_1", ascending=False).head(10)

# Grafico a barre verticali con miglioramenti visivi
fig = px.bar(
    top10,
    x="APT_NAME",
    y="FLT_ARR_1",
    title="Top 10 aeroporti per numero totale di voli in arrivo",
    labels={"APT_NAME": "Aeroporto", "FLT_ARR_1": "Totale voli in arrivo"},
    text="FLT_ARR_1"
)



fig.update_layout(
    xaxis_tickangle=30,
    yaxis_title="Totale voli in arrivo",
    xaxis_title="Aeroporto",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(
        tickformat=",",  # formatta i numeri con la virgola come separatore delle migliaia
        nticks=10         # forza la divisione in più tick
    ),
    margin=dict(t=70, b=120, l=60, r=40),  # margini più spaziosi
    height=600
)

fig.show()

In [6]:
# Calcola il totale dei voli in partenza per aeroporto
departures = df.groupby(["APT_ICAO", "APT_NAME"])["FLT_DEP_1"].sum().reset_index()

# Seleziona i primi 10 aeroporti
top10_departures = departures.sort_values("FLT_DEP_1", ascending=False).head(10)

# Crea il grafico a barre
fig = px.bar(
    top10_departures,
    x="APT_NAME",
    y="FLT_DEP_1",
    title="Top 10 aeroporti per numero totale di voli in partenza",
    labels={"APT_NAME": "Aeroporto", "FLT_DEP_1": "Totale voli in partenza"},
    text="FLT_DEP_1"
)


fig.update_layout(
    xaxis_tickangle=30,
    yaxis_title="Totale voli in partenza",
    xaxis_title="Aeroporto",
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(
        tickformat=",",
        nticks=10
    ),
    margin=dict(t=70, b=120, l=60, r=40),
    height=600
)

fig.show()

In [19]:
# Calcola arrivi e partenze totali
totali = df.groupby(["APT_ICAO", "APT_NAME"])[["FLT_ARR_1", "FLT_DEP_1"]].sum().reset_index()
totali["TOT_VOLI"] = totali["FLT_ARR_1"] + totali["FLT_DEP_1"]
top10 = totali.sort_values("TOT_VOLI", ascending=False).head(10)

# Riformatta per grafico
df_long = top10.melt(
    id_vars=["APT_NAME"],
    value_vars=["FLT_ARR_1", "FLT_DEP_1"],
    var_name="Tipo_Volo",
    value_name="Totale"
)
df_long["Tipo_Volo"] = df_long["Tipo_Volo"].replace({
    "FLT_ARR_1": "Arrivi",
    "FLT_DEP_1": "Partenze"
})

# Calcolo del range Y personalizzato
max_val = df_long["Totale"].max()
min_val = df_long["Totale"].min()
padding = (max_val - min_val) * 0.1  # aggiunge un po' di spazio

# Grafico a barre raggruppate con scala Y più stretta
fig = px.bar(
    df_long,
    x="APT_NAME",
    y="Totale",
    color="Tipo_Volo",
    barmode="group",
    title="Arrivi vs Partenze nei 10 aeroporti con più traffico",
    labels={"APT_NAME": "Aeroporto", "Totale": "Numero voli"}
)

fig.update_layout(
    yaxis=dict(range=[min_val - padding, max_val + padding]),
    xaxis_tickangle=30,
    margin=dict(t=70, b=120, l=60, r=40),
    height=600
)

fig.show()